In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import r2_score
from data_utils import load_info, create_dataloaders, load_preprocessed_data

In [35]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(NeuralNetwork, self).__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.LeakyReLU(negative_slope=0.01))
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [36]:
def l1_regularization(model, l1_lambda):
    l1_norm = sum(p.abs().sum() for p in model.parameters())
    return l1_lambda * l1_norm

def train(model, train_loader, valid_loader, criterion, optimizer, epochs, patience, l1_lambda):
    best_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.float(), targets.float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets) + l1_regularization(model, l1_lambda)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in valid_loader:
                inputs, targets = inputs.float(), targets.float()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
        
        val_loss /= len(valid_loader.dataset)
        
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')
        
        
        # if val_loss < best_loss:
        #     best_loss = val_loss
        #     patience_counter = 0
        # else:
        #     patience_counter += 1
        
        # if patience_counter >= patience:
        #     print("Early stopping triggered")
        #     break

def test(model, test_loader):
    model.eval()
    targets_list = []
    outputs_list = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.float(), targets.float()
            outputs = model(inputs)
            
            targets_list.append(targets.numpy())
            outputs_list.append(outputs.numpy())

    outputs_list = np.concatenate(outputs_list)
    targets_list = np.concatenate(targets_list)

    r2 = r2_score(targets_list, outputs_list)
    print(f'R² score: {r2}\n\n')

In [37]:
input_data, target_data = load_preprocessed_data()
firm_info, _ = load_info()
train_loader, valid_loader, test_loader, _ = create_dataloaders(
    input_data, target_data, firm_info,
    train_date='2005-01-01', valid_date='2010-01-01', test_date='2015-11-01', batch_size=2000)

In [38]:
print(f'Input data shape is{input_data.shape}')
print(f'Target data shape: {target_data.shape}')
print(f'Train loader length: {len(train_loader)}')
print(f'Valid loader length: {len(valid_loader)}')
print(f'Test loader length: {len(test_loader)}')

first_batch = next(iter(train_loader))
first_batch = next(iter(valid_loader))
first_batch = next(iter(test_loader))

print(f"Train loader: Batch size = {len(first_batch[0])}, Features = {first_batch[0][0].shape}")
print(f"Valid loader: Batch size = {len(first_batch[0])}, Features = {first_batch[0][0].shape}")
print(f"Test loader: Batch size = {len(first_batch[0])}, Features = {first_batch[0][0].shape}")

Input data shape is(576574, 252)
Target data shape: (576574, 3)
Train loader length: 114
Valid loader length: 52
Test loader length: 57
Train loader: Batch size = 2000, Features = torch.Size([250])
Valid loader: Batch size = 2000, Features = torch.Size([250])
Test loader: Batch size = 2000, Features = torch.Size([250])


In [39]:
# Hyperparameters setting
input_dim = input_data.shape[1] - 2
# hidden_layers_list = [[128, 64, 32], [64, 32, 16], [32, 16, 8], [128, 64], [64, 32], [32, 16], [16, 8], [128], [64], [32], [16], [8]]
hidden_layers_list = [[128, 64, 32], [64, 32, 16], [32, 16, 8], [128, 64], [64, 32], [32, 16], [16, 8], ]
output_dim = 1
learning_rates = [0.01, 0.001]
epochs = 100
patience = 10
l1_lambdas = [1e-5, 1e-3]

In [40]:
for hidden_layers in hidden_layers_list:
    for learning_rate in learning_rates:
        for l1_lambda in l1_lambdas:
            print(f"Training with hidden_layers={hidden_layers}, learning_rate={learning_rate}, l1_lambda={l1_lambda}\n")
            model = NeuralNetwork(input_dim, hidden_layers, output_dim)
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            train(model, train_loader, valid_loader, criterion, optimizer, epochs, patience, l1_lambda)
            test(model, test_loader)

Training with hidden_layers=[128, 64, 32], learning_rate=0.01, l1_lambda=1e-05

Epoch 1/100, Train Loss: 0.034477, Val Loss: 0.026118
Epoch 2/100, Train Loss: 0.029705, Val Loss: 0.027166
Epoch 3/100, Train Loss: 0.028963, Val Loss: 0.026514
Epoch 4/100, Train Loss: 0.028776, Val Loss: 0.026187
Epoch 5/100, Train Loss: 0.028656, Val Loss: 0.027865
Epoch 6/100, Train Loss: 0.028518, Val Loss: 0.028366
Epoch 7/100, Train Loss: 0.028466, Val Loss: 0.028261
Epoch 8/100, Train Loss: 0.028499, Val Loss: 0.027362
Epoch 9/100, Train Loss: 0.028492, Val Loss: 0.027642
Epoch 10/100, Train Loss: 0.028486, Val Loss: 0.027027
Epoch 11/100, Train Loss: 0.028406, Val Loss: 0.028156
Epoch 12/100, Train Loss: 0.028452, Val Loss: 0.027594
Epoch 13/100, Train Loss: 0.028538, Val Loss: 0.028360
Epoch 14/100, Train Loss: 0.028458, Val Loss: 0.029835
Epoch 15/100, Train Loss: 0.028383, Val Loss: 0.029104
Epoch 16/100, Train Loss: 0.028430, Val Loss: 0.028463
Epoch 17/100, Train Loss: 0.028450, Val Loss: 0.0